In [1]:
%load_ext autoreload
%autoreload 2

import casadi as ca
from autodiff_sys import Sys
from helper_fns import *

# Parameters for CL robot, desired position -> velocity
M = ca.MX.sym('M')
B = ca.MX.sym('B')
Kp = ca.MX.sym('Kp')
Kd = ca.MX.sym('Kd')
R = Sys([Kd,Kp,0],[M,B,Kp])

# Robt delay
Delay = Sys([1],[1])
#Delay = pade(2e-3, 2)

# Feedforward control
Kl = ca.MX.sym('Kl')
Cff = Sys([Kl, 0], [1])

# Feedback control + payload comp
B_real  = ca.MX.sym('B_real')
a_vel   = ca.MX.sym('a_vel')
P_hat   = ca.MX.sym('P_hat')
a_acc   = ca.MX.sym('a_acc')

# These TFs are from robot velocity
C_s_inv = Sys([B_real*a_vel, 0],[1, a_vel, 0]) # Rob vel to desired position
P_hat_Oa_s_inv = Sys([-P_hat*a_acc*a_vel, 0, 0],[1, a_acc+a_vel, a_acc*a_vel, 0])


Ma = ca.MX.sym('Ma')
Ba = ca.MX.sym('Ba')
A = Sys([1], [Ma, Ba, 0])

p = [M, B, Kp, Kd] # fixed params
x = [Kl, Ma, Ba, B_real, a_vel, P_hat, a_acc]   # dec variables
vars = [*p, *x]    # list of all the variables, useful for making casadi functions
x_names = [str(v) for v in x]

##### Building some SYSTEMS ######
# OL from F to v
G = R*Delay*(1+Cff)*A

# Feedback from payload comp and control
D = R*Delay*C_s_inv + G*P_hat_Oa_s_inv

# Fd to v with feedback
G_re, _ = (G/(1+D)).get_re_im(vars)

# Environment contact dynamics
K_env = ca.MX.sym('K_env')
M_env = ca.MX.sym('M_env')
E = Sys([M_env, 0, K_env], [1, 0])

G_cl = G/(1+D+E*G)

# Noise TFs
noise_area_of_concern = Sys([100, 0], [1, 100])
G_nf = Cff*A*noise_area_of_concern
G_nx = (C_s_inv + Cff*A*(P_hat_Oa_s_inv))*noise_area_of_concern

In [ ]:
# Optimize!

# Initial values and bounds
p0  = [5, 120, 400, 100] # numerical value of params
x0  = [0.02, 12,  1500,  0.01, 20, 18, 10]  # initial guess at dec vars
vars0 = [*p0, *x0]
ubx = [0.06,  30, 3000,  1.0,  60,  40,  60]  # upper bound
lbx = [ 0.0,   5,  100,  0.0,  0.0,  0.0, 0.0]# lower bound

# Design bounds; can be tuned
noise_bound_f = 5e-3 # 1e-3 is more realistic, but let's see
noise_bound_x = 1e-1 # 1e-3 is more realistic, but let's see
passivity_bound = -1e-2
rt_stability_bound = 1e-8
h2_solver = 'lapackqr' # scipy or lapackqr or ...

# OBJECTIVE: J will be minimized, to maximize admittance F->v
J = -G.h2(sol = h2_solver) 


# CONSTRAINTS
g = []
ubg = []
lbg = []

# OL Stability:
rt = routh_table(G.den)
for r_i in range(1, len(rt)):
    g0 =ca.Function('test', vars, [rt[r_i]])
    #print('OL RT at vars0: {}'.format(g0(*vars0)))
    g.append(rt[r_i]/g0(*vars0))
    ubg.append(np.Inf)
    lbg.append(rt_stability_bound) 


# Contact Stability:
#M_en = 12
for K_en in [5e3, 5e5]:
    #Stab
    #E = Sys([M_en, 0, K_en], [1, 0])
    E = Sys([K_en], [1, 0])
    G_cl = G/(1+D+E*G)
    rt = routh_table(G_cl.den)
    for r_i in range(1, len(rt)):
        g0 =ca.Function('test', vars, [rt[r_i]])
        #print('CL RT at vars0: {}'.format(g0(*vars0)))
        g.append(rt[r_i]/g0(*vars0))
        ubg.append(np.Inf)
        lbg.append(rt_stability_bound)    
'''
'''
#Noise
g.append(G_nf.h2(sol = h2_solver))
ubg.append(noise_bound_f)
lbg.append(-np.Inf)
    
g.append(G_nx.h2(sol = h2_solver))
ubg.append(noise_bound_x)
lbg.append(-np.Inf)

for om in np.logspace(0.1, 4, num = 50):
    imag_pt = G_re(om, *p0, *x)
    g.append(imag_pt)
    lbg.append(passivity_bound)
    ubg.append(np.Inf)
       
opts = { #'ipopt.tol': 1e-3,
         'ipopt.max_iter': 200,
         #'ipopt.constr_viol_tol': 1e-9,
         'ipopt.print_level': 5,
         'ipopt.derivative_test': 'first-order',         # Check the gradient
         'ipopt.mu_strategy': 'monotone',                # Determines which barrier parameter update strategy is to be used. Default "monotone", can choose 'adaptive' instead
         'ipopt.mu_init' : 1e2,                          # Initial value for the barrier parameter (mu). It is only relevant in the monotone, Fiacco-McCormick version of the algorithm.
         'ipopt.bound_relax_factor' : 0,                 # Relaxes constraints, default 1e-8, can cause problems here, eg if params go negativ
         'ipopt.expect_infeasible_problem' : 'yes',     # Starts ipopt to solve for feasiblity (i.e. satisfy constraints)
       }
        
# J is objective, w decision vars, g constraints (subject to ubg and lbg)
prob = dict(f = J, x = ca.vertcat(*x), g = ca.vertcat(*g), p = ca.vertcat(*p))
solver = ca.nlpsol('solver', 'ipopt', prob, opts)
args = dict(x0 = x0, lbx = lbx, ubx = ubx, ubg = ubg, lbg = lbg, p = p0)
sol = solver(**args)
x_opt = sol['x'].full().tolist()
for val, nam in zip(x_opt, x_names):
    print('{:7s} : {:.3f}'.format(nam, val[0]))

G.nyquist(vars, [*p0, *sol['x'].full()])

OL RT at vars0: 8940
OL RT at vars0: 180773
OL RT at vars0: 600000
Routh Table has leading sym 0; interrupting, may not be complete
CL RT at vars0: 5.67e+08
CL RT at vars0: 9.51642e+10
CL RT at vars0: 8.77369e+12
CL RT at vars0: 5.02351e+14
CL RT at vars0: 1.87963e+16
CL RT at vars0: 4.8085e+17
CL RT at vars0: 8.7679e+18
CL RT at vars0: 1.17412e+20
CL RT at vars0: 1.17447e+21
CL RT at vars0: 8.83424e+21
CL RT at vars0: 4.96503e+22
CL RT at vars0: 2.03666e+23
CL RT at vars0: 5.86955e+23
CL RT at vars0: 1.11375e+24
CL RT at vars0: 1.152e+24
Routh Table has leading sym 0; interrupting, may not be complete
CL RT at vars0: 5.67e+08
CL RT at vars0: 9.85246e+10
CL RT at vars0: 1.03704e+13
CL RT at vars0: 8.31487e+14
CL RT at vars0: 5.53777e+16
CL RT at vars0: 2.71693e+18
CL RT at vars0: 8.73819e+19
CL RT at vars0: 1.8786e+21
CL RT at vars0: 2.82947e+22
CL RT at vars0: 3.05036e+23
CL RT at vars0: 2.34045e+24
CL RT at vars0: 1.24373e+25
CL RT at vars0: 4.4063e+25
CL RT at vars0: 9.82081e+25
CL 

  67 -4.0686080e-03 2.03e-02 2.11e-01  -5.0 1.32e+01    -  1.00e+00 9.77e-04h 11
  68 -4.0670095e-03 1.94e-02 3.58e-01  -5.0 4.33e+01    -  1.59e-01 6.25e-02h  5
  69 -4.0664984e-03 1.91e-02 3.50e-01  -5.0 3.90e+01    -  6.45e-01 1.56e-02h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70 -4.0664331e-03 1.90e-02 3.49e-01  -5.0 1.43e+01    -  5.94e-01 9.77e-04h 11
  71 -4.0659405e-03 1.87e-02 3.43e-01  -5.0 3.34e+01    -  7.31e-01 1.56e-02h  7
  72 -4.0658776e-03 1.87e-02 3.42e-01  -5.0 1.39e+01    -  6.26e-01 9.77e-04h 11
  73 -4.0653978e-03 1.84e-02 3.33e-01  -5.0 3.05e+01    -  7.69e-01 1.56e-02h  7
  74 -4.0043402e-03 2.15e-01 1.81e+01  -5.0 1.35e+01    -  6.51e-01 1.00e+00w  1
  75 -4.0093917e-03 3.72e-01 1.68e+01  -5.0 3.94e+01  -3.9 1.84e-01 1.08e-01w  1


In [82]:

# Scale the objective
J_fn = ca.Function('J_fn', [*vars], [J])
J0 = J_fn(*p0, *x0)
#J *= 1/ca.fabs(J0)
#J_grad_fn = ca.Function('J_grad_fn', [*vars], [ca.gradient(J, ca.horzcat(*x))])
print('Objective: {}'.format(J0))
#print('Gradient:  {}'.format(J_grad_fn(*p0, *x0)))

lyap = LyapSolver('testerino', 2)
print(lyap.get_forward(1,'fwd',[],[],{}))
print(lyap.get_forward(1,'fwd',[],[],{}).has_derivative())


Getting forward deriv with nfwd 1
fwd:(i0[2x2],i1[2x2],i2[2x2],i3[2x2],i4[2x2])->(o0[2x2]) CallbackInternal
Getting forward deriv with nfwd 1


AttributeError: 'Callback' object has no attribute 'has_derivative'